In [54]:
pip install noisereduce

In [63]:
import librosa
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import noisereduce as nr

In [70]:
import noisereduce as nr

# 데이터 로드 및 Spectrogram 추출 함수
def load_and_extract_features(file_paths):
    spectrograms = []
    for file_path in file_paths:
        waveform, sample_rate = librosa.load(file_path, sr=None)
        spectrogram = librosa.feature.melspectrogram(y=waveform, sr=sample_rate)
        spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
        resized_spectrogram = np.resize(spectrogram, (128, 128, 1))  # CNN 모델에 맞게 크기 조정
        spectrograms.append(resized_spectrogram)
    return np.array(spectrograms)

# CNN 모델 생성 함수
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [71]:
# 데이터 준비
train_file_paths_1 = [f"/content/{str(i).zfill(3)}.wav" for i in range(1, 21)]
test_file_paths_1 = [f"/content/{str(i).zfill(3)}.wav" for i in range(21, 31)]

train_file_paths_2 = [f"/content/T{i:03d}.wav" for i in range(1, 11)]
test_file_paths_2 = [f"/content/T{i:03d}.wav" for i in range(11, 28)]

train_file_paths = train_file_paths_1 + train_file_paths_2
test_file_paths = test_file_paths_1 + test_file_paths_2

In [74]:
data = load_and_extract_features(train_file_paths)
labels = [1] * 20 + [0] * 10  # 정답지

In [75]:
# CNN 모델 생성
input_shape = (128, 128, 1)  # CNN 모델에 맞는 입력 형태
model = create_cnn_model(input_shape)

# 모델 훈련
model.fit(data, np.array(labels), epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6671 - accuracy: 0.5417 - val_loss: 226.2304 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 1s 725ms/step - loss: 42.3689 - accuracy: 0.8333 - val_loss: 107.6465 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 20.1793 - accuracy: 0.8333 - val_loss: 39.3441 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 7.5254 - accuracy: 0.8333 - val_loss: 10.0152 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 1.9833 - accuracy: 0.8333 - val_loss: 0.4763 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 1s 972ms/step - loss: 0.8363 - accuracy: 0.1667 - val_loss: 0.9559 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 1s 727ms/step - loss: 0.4439 - accuracy: 0.8750 - val_loss: 6.3094 - val_accuracy: 0.0000e+00


In [ ]:
# 특정 음성 파일의 물소리 판별
def predict_sound(file_path):
    # 파일 로드 및 전처리
    waveform, sample_rate = librosa.load(file_path, sr=None)
    spectrogram = librosa.feature.melspectrogram(y=waveform, sr=sample_rate)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    resized_spectrogram = np.resize(spectrogram, (128, 128))

    # 모델에 입력 형태에 맞게 차원 재구성
    input_data = resized_spectrogram[np.newaxis, ..., np.newaxis]

    # 예측
    prediction = model.predict(input_data)

    # 결과 출력
    if prediction > 0.8:
        print(f"The sound in {file_path} is classified as 'water sound'.")
    else:
        print(f"The sound in {file_path} is classified as 'non-water sound'.")

In [ ]:
# 모델 예측
new_spectrograms = generate_spectrograms(test_file_paths)
preprocessed_new_spectrograms = preprocess_spectrograms(new_spectrograms)

predictions = model.predict(np.array(preprocessed_new_spectrograms))
print(predictions)

In [82]:
for file_path, prediction in zip(test_file_paths, predictions):
    if prediction >= 0.89:
        print(f"The sound in {file_path} is classified as 'water sound'.")
    else:
        print(f"The sound in {file_path} is classified as 'non-water sound'.")

The sound in /content/021.wav is classified as 'water sound'.
The sound in /content/022.wav is classified as 'water sound'.
The sound in /content/023.wav is classified as 'water sound'.
The sound in /content/024.wav is classified as 'water sound'.
The sound in /content/025.wav is classified as 'water sound'.
The sound in /content/026.wav is classified as 'non-water sound'.
The sound in /content/027.wav is classified as 'water sound'.
The sound in /content/028.wav is classified as 'water sound'.
The sound in /content/029.wav is classified as 'water sound'.
The sound in /content/030.wav is classified as 'water sound'.
The sound in /content/T011.wav is classified as 'non-water sound'.
The sound in /content/T012.wav is classified as 'non-water sound'.
The sound in /content/T013.wav is classified as 'non-water sound'.
The sound in /content/T014.wav is classified as 'non-water sound'.
The sound in /content/T015.wav is classified as 'non-water sound'.
The sound in /content/T016.wav is classif

In [84]:
model.save('water_saver.keras')  # 'water_saver.keras' 파일로 저장